In [38]:
import praw # Python Reddit API Wrapper
#Praw documentation: https://praw.readthedocs.io/en/stable/index.html

# Create a Reddit instance
# This is a read-only instance since we only need to read data
# We don't need it to be authenticated since we aren't posting anything
reddit = praw.Reddit(
    client_id='51f3IBDpHGRzaGh0FGj0ng',
    client_secret=	'C-mvx1K5Flj_4kM-Grc9FLT2_bHxag',
    user_agent='TradingAnalysisApp by /u/Ok_Ability_9624'
)


In [39]:
print(reddit.read_only)

True


In [40]:
# Fetch and print the titles of the top 10 hot posts in the 'test' subreddit
for submission in reddit.subreddit("test").hot(limit=10):
    print(submission.title)

Some test commands
Test me123
Test
just a test
My first text post with title "The great cookie caper"
Mikes number 1 enemey
Rage
please dont be mean to isaam :(
Test
ahhhh


In [41]:
subreddit = reddit.subreddit("wallstreetbets")
print(subreddit.display_name)  
print(subreddit.title)
print(subreddit.description)


wallstreetbets
wallstreetbets
The rules and submission guidelines are maintained on new Reddit so be sure to check them and make sure you're up to date.

* Read the [rules](https://www.reddit.com/r/wallstreetbets/about/rules)

* Read the [comment and submission guide](https://www.reddit.com/r/wallstreetbets/wiki/contentguide)

* Read the [FAQ](https://www.reddit.com/r/wallstreetbets/wiki/faq) if you're new to both wallstreetbets and trading.

---
**Join the discord**

[WSB Discord](https://discord.gg/wsbverse)

**filter by flairs**

^Navigate ^WSB |^We ^recommend ^best ^daily ^DD
:--|:--     
**DD** | [All](https://ns.reddit.com/r/wallstreetbets/search?sort=new&restrict_sr=on&q=flair%3ADD) / [**Best Daily**](https://ns.reddit.com/r/wallstreetbets/search?sort=top&q=flair%3ADD&restrict_sr=on&t=day) / [Best Weekly](https://ns.reddit.com/r/wallstreetbets/search?sort=top&q=flair%3ADD&restrict_sr=on&t=week)
**Discussion** | [All](https://ns.reddit.com/r/wallstreetbets/search?sort=new&restric

In [45]:

postData = []
# Fetch the top 100000 hot posts in the 'wallstreetbets' subreddit
for submission in subreddit.hot(limit=100000):
        if(submission.selftext == ''): #skip any posts that contain no text
            continue
        postData.append({
        'title': submission.title,
        'score': submission.score,  # upvotes
        'id': submission.id,
        'url': submission.url,
        'comms_num': submission.num_comments,
        'created': submission.created_utc,
        'body': submission.selftext
        })

In [46]:
import pandas as pd

df = pd.DataFrame(postData)
df



,title,score,id,url,comms_num,created,body
0,"What Are Your Moves Tomorrow, October 22, 2024",81,1g8z4d2,https://www.reddit.com/r/wallstreetbets/commen...,1825,1.729541e+09,This post contains content not supported on ol...
1,HOW DO I TELL MY PARENTS / FIANCÉ I LOST ALL M...,1999,1g8tfk7,https://www.reddit.com/r/wallstreetbets/commen...,1104,1.729527e+09,"I’ll get a whole lecture if they find out, so ..."
2,All eggs in one basket since last year.,253,1g8ze2n,https://www.reddit.com/gallery/1g8ze2n,70,1.729541e+09,All long term investments (started Aug 2021). ...
3,"Decade of Big S&P 500 Gains Is Over, Goldman S...",1132,1g8nu3u,https://finance.yahoo.com/news/p-500-decade-bi...,285,1.729512e+09,>The S&P 500 Index is expected to post an annu...
4,What happens when a stock falls out of the S&P...,298,1g8u2a5,https://www.investopedia.com/delisting-a-stock...,45,1.729528e+09,ni
...,...,...,...,...,...,...,...
341,4600% on some lotto tickets. Thanks AFRM 🫡,130,1g1o5lc,https://www.reddit.com/gallery/1g1o5lc,29,1.728693e+09,Too bad I sold 80 contracts for a 40% gain 😭😭
342,Tesla short results,831,1g1afzh,https://www.reddit.com/gallery/1g1afzh,76,1.728655e+09,The shit show paid out as expected
343,Being late pays off ig,44,1g1qzqz,https://i.redd.it/grk1wjn6r8ud1.jpeg,4,1.728702e+09,LITERALLY ALMOST NEVER CAN YOU GET INTO A TRAD...
344,Who said Options couldn't be profitable?,9,1g1xlwx,https://i.redd.it/amv2fztp3bud1.png,21,1.728731e+09,"₹ 1L = ₹ 100000 for those wondering, wanted to..."
